In [1]:
import json
import os
import random

import numpy as np
import pandas as pd
import pycm

In [2]:
from dataset import Dataset, LabelManager
from filtration import FilterManager, FilterBlackAndWhite, FilterHSV, FilterFocusMeasure

In [3]:
from model_analysis.by_region_analysis_job import WeightRatioAnalysisJob

In [4]:
try:
    imgs_path = "/workspaces/dev-container/testing/data/whole_slide_images/train"
    dataset = Dataset(
        imgs_path,
        labels=LabelManager(
            imgs_path,
            label_postprocessor=lambda path: int(os.path.basename(path))
        ),
        filtration=FilterManager([
            FilterBlackAndWhite(),
            FilterHSV(),
            FilterFocusMeasure()
        ])
    )
    analysis_job = WeightRatioAnalysisJob(
        dataset = dataset,
        region_prediction_callback = lambda *args, **kwargs: np.arange(args[0].shape[0])
    )
    analysis_job.get_region_predictions_from_model(loadingbars=True, loadingbars_ascii=True)
finally:
    print('--> deleting dataset bc h5 file locking')
    try:
        del(dataset)
    except Exception:
        print('--> failed to delete dataset')

preprocessing file /workspaces/dev-container/testing/data/whole_slide_images/train/0/84440T_004.tif
preprocessing file /workspaces/dev-container/testing/data/whole_slide_images/train/1/84429T_003.tif


Getting region predictions:   0%|          | 0/2 [00:00<?, ?it/s]

--> deleting dataset bc h5 file locking


In [ ]:
data = pd.read_csv('model_analysis_results_weight_ratio.csv', index_col=False)
data.head(5)

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.interpolate import griddata

x = data['weight_1'] / data['weight_0']
y = data['weight_2'] / data['weight_1']
z = data['F1_Micro']


xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)

X,Y = np.meshgrid(xi,yi)

Z = griddata((x,y),z,(X,Y))

fig = go.Figure(go.Surface(x=xi,y=yi,z=Z))

fig.update_layout(
    title = "Weight Ratio Analysis: F1 Micro",
    autosize=False,
    width = 600, height=400
)

fig.show()